In [1]:
import time
from ccd.procedures import fit_procedure as __determine_fit_procedure
import numpy as np
import pandas as pd
from ccd import app, math_utils, qa, __check_inputs, __sort_dates, attr_from_str
from datetime import datetime
import importlib

import timeit

from test.shared import rainbow

In [2]:
requested_ubids = ['LANDSAT_4/TM/SRB1', 'LANDSAT_4/TM/SRB2', 'LANDSAT_4/TM/SRB3', 'LANDSAT_4/TM/SRB4', 'LANDSAT_4/TM/SRB5', 'LANDSAT_4/TM/BTB6', 'LANDSAT_4/TM/SRB7', 'LANDSAT_4/TM/PIXELQA', 'LANDSAT_5/TM/SRB1', 'LANDSAT_5/TM/SRB2', 'LANDSAT_5/TM/SRB3', 'LANDSAT_5/TM/SRB4', 'LANDSAT_5/TM/SRB5', 'LANDSAT_5/TM/BTB6', 'LANDSAT_5/TM/SRB7', 'LANDSAT_5/TM/PIXELQA', 'LANDSAT_7/ETM/SRB1', 'LANDSAT_7/ETM/SRB2', 'LANDSAT_7/ETM/SRB3', 'LANDSAT_7/ETM/SRB4', 'LANDSAT_7/ETM/SRB5', 'LANDSAT_7/ETM/BTB6', 'LANDSAT_7/ETM/SRB7', 'LANDSAT_7/ETM/PIXELQA', 'LANDSAT_8/OLI_TIRS/SRB2', 'LANDSAT_8/OLI_TIRS/SRB3', 'LANDSAT_8/OLI_TIRS/SRB4', 'LANDSAT_8/OLI_TIRS/SRB5', 'LANDSAT_8/OLI_TIRS/SRB6', 'LANDSAT_8/OLI_TIRS/SRB7', 'LANDSAT_8/OLI_TIRS/BTB10', 'LANDSAT_8/OLI_TIRS/PIXELQA']
specs_url  = 'foo'
chips_url  = 'bar'
acquired   = '1982-01-01/2015-12-31'
x, y       = -2094585, 1682805
xind, yind = 57, 97
row, col   = yind, xind

In [3]:
rbow = rainbow(x, y, acquired, specs_url, chips_url, requested_ubids)

In [4]:
rbow

<xarray.Dataset>
Dimensions:  (t: 2809, x: 100, y: 100)
Coordinates:
  * t        (t) object '1982-11-15T18:01:50Z' '1982-11-24T17:55:58Z' ...
Dimensions without coordinates: x, y
Data variables:
    red      (t, x, y) float64 6.721e+03 6.766e+03 6.721e+03 6.632e+03 ...
    nir      (t, x, y) float64 6.73e+03 6.73e+03 6.676e+03 6.676e+03 ...
    thermal  (t, x, y) float64 2.484e+03 2.484e+03 2.484e+03 2.484e+03 ...
    green    (t, x, y) float64 6.653e+03 6.705e+03 6.653e+03 6.551e+03 ...
    swir2    (t, x, y) float64 3.457e+03 3.348e+03 3.457e+03 3.348e+03 ...
    cfmask   (t, x, y) object 224 224 224 224 224 224 224 224 224 224 224 ...
    swir1    (t, x, y) float64 4.398e+03 4.398e+03 4.435e+03 4.398e+03 ...
    blue     (t, x, y) float64 2e+04 2e+04 2e+04 2e+04 2e+04 2e+04 2e+04 ...

In [4]:
proc_params = app.get_default_params()

In [5]:
# dates
def dtstr_to_ordinal(dtstr, iso=True):
    """ Return ordinal from string formatted date"""
    _fmt = '%Y-%m-%dT%H:%M:%SZ' if iso else '%Y-%m-%d %H:%M:%S'
    _dt = datetime.strptime(dtstr, _fmt)
    return _dt.toordinal()


rainbow_date_array = np.array(rbow['t'].values)
_dates = [dtstr_to_ordinal(str(pd.to_datetime(i)), False) for i in rainbow_date_array]

# date handling with ccd.__init__::detect
dates = np.asarray(_dates)

In [6]:
# quality
_quality = np.array(rbow['cfmask'].values[:, row, col], dtype=int)

# quality handling with ccd.__init__::detect
quality = np.asarray(_quality)

In [7]:
# how bands passed to ccd.detect()
blue    = np.array(rbow['blue'].values[:, row, col])
green   = np.array(rbow['green'].values[:, row, col])
red     = np.array(rbow['red'].values[:, row, col])
nir     = np.array(rbow['nir'].values[:, row, col])
swir1   = np.array(rbow['swir1'].values[:, row, col])
swir2   = np.array(rbow['swir2'].values[:, row, col])
thermal = np.array(rbow['thermal'].values[:, row, col])

# how bands org'd within ccd.__init__::detect
spectra = np.stack((blue, green, red, nir, swir1, swir2, thermal))

In [9]:
# input validation in ccd.__init__::detect
__check_inputs(dates, quality, spectra)

In [9]:
# variable assignment in ccd.__init__::detect
indices = __sort_dates(dates)
dates   = dates[indices]
spectra = spectra[:, indices]
quality = quality[indices]

In [12]:
# how ccd.__init__::detect determines which procedure to use
t1 = datetime.now()
procedure = __determine_fit_procedure(quality, proc_params)
print("seconds to run: {}".format((datetime.now() - t1).total_seconds()))
print(procedure)

seconds to run: 0.00055
<function standard_procedure at 0x7f6bcaea3730>


In [14]:
#t1 = datetime.now()
#results = procedure(dates, spectra, fitter_fn, quality, proc_params)
#print("seconds to run: {}".format((datetime.now() - t1).total_seconds()))
print(timeit.timeit("procedure(dates, spectra, fitter_fn, quality, proc_params)", globals=globals(), number=3))

/home/cgaustin/miniconda2/envs/ccd35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


2.777985084001557
